# Scenario Report

In [ ]:
# Copyright (c) 2021 The MATCH Authors. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

%reload_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly
import plotly.graph_objects as go 
import numpy as np
from switch_model.reporting.report_functions import *
import pickle

# TODO: remove input folder = '../../MODEL_RUNS/test_PCE/inputs/hourly_90'

#get the name of the current directory to specify the scenario name and identify the output directory
scenario_name = str(Path.cwd()).split('\\')[-1]
if scenario_name == 'inputs':
    data_dir = Path.cwd() / '../outputs/'
    inputs_dir = Path.cwd() / '../inputs/'
    scenario_name = 'N/A'
else:
    data_dir = Path.cwd()/ f'../../outputs/{scenario_name}/'
    inputs_dir = Path.cwd() / f'../../inputs/{scenario_name}/'

#define formatting options/functions for outputs
pd.options.display.float_format = '{:,.2f}'.format

#allow the notebook to display plots in html report
###################################################
plotly.offline.init_notebook_mode()

print(f'Scenario Name: {scenario_name}')

In [ ]:
# load data from csvs
baseload_capacity_factors = pd.read_csv(inputs_dir / 'baseload_capacity_factors.csv')
with open(inputs_dir / 'cambium_region.txt', 'r') as file:
    cambium_region = file.read()
costs_by_gen = pd.read_csv(data_dir / 'costs_by_gen.csv')
costs_by_tp = pd.read_csv(data_dir / 'costs_by_tp.csv')
dispatch = pd.read_csv(data_dir / 'dispatch.csv')
with open(inputs_dir / 'ghg_emissions_unit.txt', 'r') as unit:
    emissions_unit = unit.read()
financials = pd.read_csv(inputs_dir / 'financials.csv')
fixed_costs = pd.read_csv(inputs_dir / 'fixed_costs.csv')
gen_build_predetermined = pd.read_csv(inputs_dir / 'gen_build_predetermined.csv')
gen_cap = pd.read_csv(data_dir / 'gen_cap.csv')
generation_projects_info = pd.read_csv(inputs_dir / 'generation_projects_info.csv')
with open(inputs_dir / 'gen_set.txt', 'r') as set_name:
    gen_set = set_name.read()
load_balance = pd.read_csv(data_dir / 'load_balance.csv')
nodal_prices = pd.read_csv(inputs_dir / 'nodal_prices.csv')
periods = pd.read_csv(inputs_dir / 'periods.csv')
with open((data_dir / 'results.pickle'), 'rb') as file:
    results = pickle.load(file)
# load RA data if modeled
try:
    ra_summary = pd.read_csv(data_dir / 'RA_summary.csv')
    ra_exists = True
except FileNotFoundError:
    ra_exists = False
    ra_summary = pd.DataFrame()
rec_value = pd.read_csv(inputs_dir / 'rec_value.csv')

system_power = pd.read_csv(data_dir / 'system_power.csv')
timestamps = pd.read_csv(inputs_dir / 'timepoints.csv', parse_dates=['timestamp'], usecols=['timepoint_id','timestamp'])
variable_capacity_factors = pd.read_csv(inputs_dir / 'variable_capacity_factors.csv')
try:
    storage_builds = pd.read_csv(data_dir / 'storage_builds.csv')
    storage_cycle_count = pd.read_csv(data_dir / 'storage_cycle_count.csv')
    storage_dispatch = pd.read_csv(data_dir / 'storage_dispatch.csv')
    # there may not be any hybrid storage
    try:
        hybrid_pair = hybrid_pair_dict(generation_projects_info)
    except KeyError:
        hybrid_pair = dict()
    storage_exists = True
except:
    storage_exists = False
    storage_builds = pd.DataFrame()
    storage_cycle_count = pd.DataFrame()
    storage_dispatch = pd.DataFrame()
    hybrid_pair = dict()

model_year = periods.loc[0,'period_start']
base_year = financials.loc[0,'base_financial_year']

technology_color_map = {
 'Small Hydro':'Purple',
 'Consumed Small Hydro':'Purple',
 'Excess Small Hydro':'Plum',
 'Onshore Wind':'DeepSkyBlue',
 'Consumed Onshore Wind':'DeepSkyBlue',
 'Excess Onshore Wind':'LightSkyBlue',
 'Offshore Wind':'Navy',
 'Consumed Offshore Wind':'Navy',
 'Excess Offshore Wind':'MediumSlateBlue',
 'Solar PV':'Gold',
 'Hybrid Solar PV':'Gold',
 'Consumed Solar PV':'Gold',
 'Excess Solar PV':'Yellow',
 'CSP':'Orange',
 'Geothermal':'Sienna',
 'Consumed Geothermal':'Sienna',
 'Storage':'Green',
 'Storage Discharge':'Green',
 'Hybrid Storage':'Green',
 'Grid Energy':'Red',
 '(?)':'Black'}

# Renewable Energy Goal

## Portfolio Renewable Percentage

In [ ]:
  
print(f'Time-coincident renewable percentage: {format_percent(hourly_renewable_percentage(load_balance))}')
print(f'Annual volumetric renewable percentage: {format_percent(annual_renewable_percentage(load_balance))}')

## Sensitivity Analysis
The following shows how well the selected portfolio would perform in specific resource years, considering the full intermittency and variability of wind and solar resources. 

In [ ]:
sensitivity_table = run_sensitivity_analysis(gen_set, gen_cap, dispatch, generation_projects_info, load_balance, storage_builds, storage_exists)

## Carbon footprint of delivered energy

In [ ]:
cambium_mid_model_year = load_cambium_data(scenario='MidCase', year=model_year, region=cambium_region)
total_emissions = calculate_emissions(dispatch, generation_projects_info, system_power, load_balance, cambium_mid_model_year, emissions_unit)
print(f'Total Annual Emissions: {total_emissions["Total Emission Rate"].sum().round(1):,} {emissions_unit.split("/")[0]}')
print(f'Delivered Emission Factor: {total_emissions["Delivered Emission Factor"].mean().round(3)} {emissions_unit}')

In [ ]:
build_hourly_emissions_heatmap(total_emissions, emissions_unit).show()

# Generator Portfolio

The sunburst chart describes the built portfolio at various levels of detail, which shows how the outer rings relate to the inner rings
- Inner circle: contract status (contracted or additional project)
- Middle ring: technology type (e.g. solar, wind, ...)
- Outer ring: specific project name

For example, individual projects in the outer ring belong to a specific technology type in the middle ring, which can either be part of the existing/contracted portfolio, or the additional portfolio.

In [ ]:
portfolio = generator_portfolio(gen_cap, gen_build_predetermined, generation_projects_info, base_year)

portfolio_sunburst = px.sunburst(portfolio, path=['Contract Status', 'Build Status','Technology','generation_project'], values='MW', color='Technology', color_discrete_map=technology_color_map,
 width=1000, height=1000,
 title='Energy Portfolio by Project Name, Technology Type, Build Status, and Contract Status (MW)')
portfolio_sunburst.update_traces(textinfo='label+value')
portfolio_sunburst.show()

## Power Content Label


In [ ]:
power_content = power_content_label(load_balance, dispatch, generation_projects_info)

delivered_energy_pie = px.pie(power_content, values='MWh', names='Source', title='Power Content of Delivered Energy (MWh)', color='Source', color_discrete_map=technology_color_map,
width=600, height=600
 )
delivered_energy_pie.update_traces(textinfo='percent+label+value')
delivered_energy_pie.show()

## Generation Cost by Project
This shows contract costs, Nodal costs, and storage revenues

In [ ]:
gen_costs = generator_costs(costs_by_gen, storage_dispatch, hybrid_pair, gen_cap, generation_projects_info, storage_exists)

generator_costs_melted = gen_costs.drop(columns=['Congestion Cost','Total Cost']).melt(id_vars='generation_project', var_name='Cost', value_name='$/MWh')

generator_cost_fig = px.bar(generator_costs_melted, 
                            title=f'Average Generator Cost per MWh Generated ({model_year}$)', 
                            x='generation_project', 
                            y='$/MWh', 
                            text='$/MWh',
                            color='Cost', 
                            category_orders={'Cost':['Energy Contract Cost', 'Capacity Contract Cost', 'Pnode Revenue', 'Delivery Cost','Congestion Cost','Storage Arbitrage Revenue']},
                            color_discrete_map={'Energy Contract Cost': 'Red',
                                                'Capacity Contract Cost': 'Orange',
                                                'Delivery Cost': 'lightblue',
                                                'Pnode Revenue':'blue', 
                                                'Congestion Cost':'Pink',
                                                'Storage Arbitrage Revenue':'purple'}).update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='black')
generator_cost_fig.update_traces(textposition='inside')
generator_cost_fig.add_scatter(x=gen_costs.generation_project, y=gen_costs['Total Cost'], mode='markers+text', text=gen_costs['Total Cost'], textposition='top center', line=dict(color='black', width=1), name='Total Cost')

generator_cost_fig.show()



## Generator Utilization

In [ ]:
utilization = calculate_generator_utilization(dispatch)
utilization

# Costs

In [ ]:
hourly_costs = hourly_cost_of_power(system_power, costs_by_tp, ra_summary, gen_cap, storage_dispatch, fixed_costs, storage_exists)
cost_table = construct_cost_table(hourly_costs, load_balance, rec_value, financials, model_year)
print(f'Model year ({model_year}) costs are discounted to present value in {base_year} using a {financials.loc[0,"discount_rate"]*100}% discount rate')
display(cost_table.set_index('Cost Component'))

# Portfolio Dispatch

In [ ]:
dispatch_by_tech, load_line, storage_charge, dispatch_fig = build_dispatch_plot(generation_projects_info, dispatch, storage_dispatch, load_balance, system_power, technology_color_map, storage_exists)
dispatch_fig.show()

## Wholesale prices

In [ ]:
build_nodal_prices_plot(nodal_prices, timestamps, generation_projects_info, model_year).show()

# Energy Storage Metrics

## Battery state of charge

In [ ]:
if storage_exists:
    build_state_of_charge_plot(storage_dispatch, storage_builds, generation_projects_info).show()
else:
    print("Storage not modeled")

## Battery Cycling and State of Charge Stats

In [ ]:
if storage_exists:
    display(construct_storage_stats_table(storage_cycle_count, storage_builds, storage_dispatch))
else:
    print("Storage not modeled")

# Monthly positions

### Month hour average dispatch

In [ ]:
build_month_hour_dispatch_plot(dispatch_by_tech, load_line, storage_charge, technology_color_map, storage_exists).show()

### Month-hour average net position

Positive values represent excess generation  
Negative values represent an open position

In [ ]:
build_open_position_plot(load_balance, storage_exists).show()

## Resource Adequacy Position

In [ ]:
if ra_exists:
    build_ra_open_position_plot(ra_summary).show()
else:
    print('Resource Adequacy was not modeled')

# Shadow Price of Energy Efficiency
The following plot shows the month-hour average value of decreasing load by 1 MW for each hour of the year

In [ ]:
try:
    calculate_load_shadow_price(results, timestamps, model_year).show()
except ValueError:
    print('Shadow prices not generated')

# Impact Metrics

## Consequential Emissions Impact

### Long Run Marginal Emissions Impact
Estimates the consequential long run impact of additional generation and storage dispatch in the portfolio for the lifetime of the contract. This metric quantifies emissions impacts resulting from changes in generator dispatch, commitment, investments, and retirements.

In [ ]:
# determine which generation is additional
addl_dispatch, addl_storage_dispatch = determine_additional_dispatch(portfolio, dispatch, storage_dispatch, storage_exists)

# load LRMER data
lrmer_data = calculate_levelized_lrmer(model_year, 20, 0.03, emissions_unit, cambium_region)

lr_impact = pd.DataFrame()
# calculate the impacts for storage and generation
lr_generation_impact = lrmer_data.mul(addl_dispatch['Total_Dispatch'], axis=0).sum()
lr_storage_impact = lrmer_data.mul(addl_storage_dispatch['Storage_Dispatch'], axis=0).sum()

lr_impact[f'Generation {emissions_unit.split("/")[0]}'] = lr_generation_impact
lr_impact[f'Storage {emissions_unit.split("/")[0]}'] = lr_storage_impact
lr_impact[f'Total {emissions_unit.split("/")[0]}'] = lr_impact.sum(axis=1)
lr_impact[f'Total {emissions_unit}'] = lr_impact[f'Total {emissions_unit.split("/")[0]}'] / (addl_dispatch['Total_Dispatch'].sum(axis=0) + addl_storage_dispatch['Storage_Dispatch'].sum(axis=0)) * -1

# append the average values
lr_impact = lr_impact.append(lr_impact.mean(axis=0).rename('Average'))

lr_impact

### Short Run Marginal Emissions Impact
This metric estimates the short-run consequential emissions impact of additional generation and storage for a single year (the model year). This metric only represents estimated imapcts on generator dispatch and commitment, and not generator build decisions. Generally, the long-run metric is more appropriate for analyzing the consequential impact of the portfolio, but this metric is provided for comparison.

In [ ]:
srmer_data = load_srmer_data(model_year, emissions_unit, cambium_region)

sr_impact = pd.DataFrame()
# calculate the impacts for storage and generation
sr_generation_impact = srmer_data.mul(addl_dispatch['Total_Dispatch'], axis=0).sum()
sr_storage_impact = srmer_data.mul(addl_storage_dispatch['Storage_Dispatch'], axis=0).sum()

sr_impact[f'Generation {emissions_unit.split("/")[0]}'] = sr_generation_impact
sr_impact[f'Storage {emissions_unit.split("/")[0]}'] = sr_storage_impact
sr_impact[f'Total {emissions_unit.split("/")[0]}'] = sr_impact.sum(axis=1)
sr_impact[f'Total {emissions_unit}'] = sr_impact[f'Total {emissions_unit.split("/")[0]}'] / (addl_dispatch['Total_Dispatch'].sum(axis=0) + addl_storage_dispatch['Storage_Dispatch'].sum(axis=0)) * -1

# append the average values
sr_impact = sr_impact.append(sr_impact.mean(axis=0).rename('Average'))

sr_impact

## Impact on the Systemwide Daily Net Demand Peak
This metric shows how additional wind and solar generation (categorized as "future" builds in the generator portfolio chart) affect the CAISO system net demand profile, specifically how the addition of these resources increase or decrease the systemwide net demand peak. Impacts are shown both based just on wind and solar generation, and for wind, solar, and storage dispatch.

In [ ]:
# use the cambium mid case for the base year
cambium_mid = load_cambium_data('MidCase', base_year, cambium_region)

# calculate system metrics
if not addl_dispatch.empty:
    display(compare_system_peaks(cambium_mid, addl_dispatch, addl_storage_dispatch))
else:
    print('No additional generators')

## Impact on Steepness of the Systemwide Daily Maximum Net Demand Ramp
This metric shows how additional wind and solar generation (categorized as "future" builds in the generator portfolio chart) affect the CAISO system net demand profile, specifically how the addition of these resources increase or decrease the steepness of the daily maximum 3 hour ramp. This ramp typically occurs in the evening as the sun goes down, and is one of the challenges of renewable integration. Impacts are shown both based just on wind and solar generation, and for wind, solar, and storage dispatch.

In [ ]:
# calculate daily 3 hour ramp
ramp_length = 3

if not addl_dispatch.empty:
    display(compare_system_ramps(cambium_mid, addl_dispatch, addl_storage_dispatch, ramp_length))
else:
    print('No additional generators')

# Assumptions

In [ ]:
pd.set_option('display.max_rows',100)
gen_assumptions = pd.read_csv(inputs_dir / 'generation_projects_info.csv', usecols=['GENERATION_PROJECT','gen_tech','gen_energy_source','ppa_energy_cost','gen_capacity_limit_mw'])
gen_assumptions = gen_assumptions[gen_assumptions['gen_energy_source'] != 'Electricity']
gen_assumptions = gen_assumptions.sort_values(by='GENERATION_PROJECT')
gen_assumptions = gen_assumptions.set_index('GENERATION_PROJECT')
display(gen_assumptions)

# Storage Assumptions

In [ ]:
if storage_exists:
    storage_assumptions = pd.read_csv(inputs_dir / 'generation_projects_info.csv', usecols=['GENERATION_PROJECT','gen_tech','gen_energy_source','ppa_capacity_cost','gen_capacity_limit_mw', 'storage_roundtrip_efficiency','storage_charge_to_discharge_ratio','storage_energy_to_power_ratio','storage_leakage_loss','storage_hybrid_generation_project','storage_hybrid_min_capacity_ratio','storage_hybrid_max_capacity_ratio'])
    #change capacity cost to $/kw-mo
    storage_assumptions['ppa_capacity_cost'] = storage_assumptions['ppa_capacity_cost'] / 12000

    storage_assumptions = storage_assumptions.rename(columns={
        'storage_roundtrip_efficiency':'RTE',
        'storage_charge_to_discharge_ratio':'charge/discharge_ratio',
        'storage_energy_to_power_ratio':'storage_hours',
        'storage_leakage_loss':'soc_leakage_loss',
        'storage_hybrid_generation_project':'paired_hybrid_gen',
        'storage_hybrid_min_capacity_ratio':'hybrid_min_capacity_ratio',
        'storage_hybrid_max_capacity_ratio':'hybrid_max_capacity_ratio',
    })

    storage_assumptions = storage_assumptions[storage_assumptions['gen_tech'] == 'Storage']
    storage_assumptions = storage_assumptions.sort_values(by='GENERATION_PROJECT')
    storage_assumptions = storage_assumptions.set_index('GENERATION_PROJECT')
    storage_assumptions = storage_assumptions[['gen_tech','ppa_capacity_cost','gen_capacity_limit_mw', 'RTE','storage_hours','charge/discharge_ratio','soc_leakage_loss','paired_hybrid_gen','hybrid_min_capacity_ratio','hybrid_max_capacity_ratio']]
    display(storage_assumptions)
else:
    print("Storage not modeled")

In [ ]:
# export scenario summary
# create an output summary table
construct_summary_output_table(scenario_name, cost_table, load_balance, portfolio, sensitivity_table, lr_impact, sr_impact, total_emissions, emissions_unit, base_year).to_csv(data_dir / 'scenario_summary.csv')